In [1]:
import sys # for automation and parallelisation
manual, scenario = (True, 'base') if 'ipykernel' in sys.argv[0] else (False, sys.argv[1])
if manual:
    %matplotlib inline


Bad key text.latex.preview in file C:\Users\marlin.arnz\AppData\Local\Continuum\miniconda3\envs\quetzal\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 123 ('text.latex.preview : False')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.2/matplotlibrc.template
or from the matplotlib source distribution

Bad key mathtext.fallback_to_cm in file C:\Users\marlin.arnz\AppData\Local\Continuum\miniconda3\envs\quetzal\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 155 ('mathtext.fallback_to_cm : True  # When True, use symbols from the Computer Modern')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.2/matplotlibrc.template
or from the matplotlib source distribution

Bad key savefig.jpeg_quality in file C:\Users\marlin.arnz\AppData\Local\Continuum\miniconda3\envs\quetzal\lib\site-packages\matplotlib\mpl-data\stylelib\

In [14]:
import pandas as pd
import numpy as np
import geopandas as gpd
from quetzal.model import stepmodel
from quetzal.io import excel
import biogeme.database as db
import biogeme.biogeme as bio
import biogeme.models as models
from biogeme import expressions as ex

In [3]:
model_path = '../model/' + scenario + '/'
input_path = '../input/'
input_static_path = '../input_static/'
output_path = '../output/'

In [4]:
# Load scenario parameters
params = excel.read_var(file='../input/parameters.xls', scenario=scenario)

# OD matrix generation

## Needs zone attributes and destination choice model results

## Saves volumes table

In [5]:
# Transport demand segments
segments = [s.strip() for s in params['general']['demand_segments'].split(';')]
purposes = ['commuting', 'business', 'education', 'buy/execute', 'leisure', 'accompany']
purp_n_dict = dict(zip(purposes, [1,2,3,4,6,7]))

In [6]:
# Load zones
sm = stepmodel.read_json(model_path + 'de_zones')
# Sum of trips per day from generation step (million)
sm.zones[[str((s, 'generation')) for s in segments]].sum().sum() / 1e6 / 365

170.00337643647052

## Calculate probabilities of inner/inter-zonal choice

In [7]:
# Load employment data
employment = pd.read_csv(input_static_path + 'spatial_employed_persons_2017_eurostat.csv', encoding='latin-1')
employment['Value'] = (pd.to_numeric(employment['Value'].str.replace(',', '')) * 1000).astype(int)
# Also, load NUTS-names
nuts = gpd.read_file(input_static_path + "spatial_NUTS_RG_01M_2016_4326.geojson")
nuts = nuts[(nuts["CNTR_CODE"]=="DE") & (nuts["LEVL_CODE"]==3)]
# Map differing names of zones to names in employment data
name_dict = {'Burgenlandkreis (DE)': 'Burgenlandkreis',
             'Dillingen an der Donau': 'Dillingen a.d. Donau',
             'Mühldorf am Inn': 'Mühldorf a. Inn',
             'Neumarkt in der Oberpfalz': 'Neumarkt i. d. OPf.',
             'Neustadt an der Aisch-Bad Windsheim': 'Neustadt a. d. Aisch-Bad Windsheim',
             'Neustadt an der Waldnaab': 'Neustadt a. d. Waldnaab',
             'Pfaffenhofen an der Ilm': 'Pfaffenhofen a. d. Ilm',
             'Weiden in der Oberpfalz, Kreisfreie Stadt': 'Weiden i. d. Opf, Kreisfreie Stadt',
             'Wunsiedel im Fichtelgebirge': 'Wunsiedel i. Fichtelgebirge'}
employment['GEO'].replace(name_dict, inplace=True)
zone_dict = employment.set_index('GEO')['Value'].to_dict()
name_dict = nuts.set_index('NUTS_ID')['NUTS_NAME'].to_dict()
sm.zones['employment'] = sm.zones['NUTS_ID'].map(name_dict).map(zone_dict) * sm.zones['pop_share']

In [8]:
# Fill POI values
pois = pd.read_csv(input_path + 'spatial_num_pois_raw.csv', index_col='index')
cats = pd.read_excel(input_path + 'spatial_OSM_POI_list.xlsx', sheet_name='categories')
cats['label'] = (cats['key'] + ' ' + cats['value'].fillna('')).str.strip()
for category, columns in cats.loc[cats['category'].notna()
                                 ].groupby('category').agg(
                                {'label': list})['label'].items():
    sm.zones[category] = sm.zones['FID'].map(pois[columns].sum(axis=1)).fillna(0)

In [9]:
# Load inter-zonal composite cost from mode choice step
cc = pd.read_csv(output_path + scenario + '/mode_choice_od_composite_cost.csv')
cc.set_index(['origin', 'destination'], inplace=True)
# Rename segments to integer values
cc.columns = pd.MultiIndex.from_tuples(
    [(seg.split('_')[0], {'no': 0, 'car': 1}[seg.split('_')[1]])
     for seg in cc.columns],
    names=['purpose_model', 'car_av'])
# Reshape the table into a mergable format
# Use the mean CC to all destinations as accessibility
cc = cc.unstack('destination').stack('purpose_model').stack('car_av').mean(axis=1)
cc = cc.unstack('purpose_model')

In [10]:
# Load inner/inter choice model results
betas = pd.read_csv(input_path + 'inner-inter_betas.csv', index_col=0)

In [16]:
# Simulate the probabilities from the zones table using biogeme
inner_probs = pd.DataFrame(index=sm.zones.index)
car_names = {0: '_no_car', 1: '_car'}
for car in [0,1]:
    for p in purposes:
        # Add accessibility for this segment to database
        sm.zones['acc_'+p] = sm.zones.merge(cc.xs(car, level='car_av'),
                                            how='left', left_index=True, right_index=True
                                           )[p]
        # Create the database
        zones_db = db.Database('zones', sm.zones[
            ['employment', 'population', 'area', 'acc_'+p]
            + list(cats.loc[cats['category'].notna(), 'category'].unique())])
        globals().update(zones_db.variables)
        
        # Define utility formulations as in cal22
        # Define parameters
        ASC_0 = ex.Beta('ASC_0', 0, None, None, 1)
        ASC_1 = ex.Beta('ASC_1', 0, None, None, 0)
        b_pop = ex.Beta('b_pop', 0, None, None, 0)
        b_attr = ex.Beta('b_attr', 0, None, None, 0)
        b_acc = ex.Beta('b_acc', 0, None, None, 0)

        # Define the utility formulation by purpose
        if p == 'commuting':
            V= {0:
                ASC_0
                + ex.log(1+employment)*b_attr
                + ex.log((1+population)/area)*b_pop,
                1:
                ASC_1
                + acc_commuting * b_acc}
        elif p == 'business':
            V= {0:
                ASC_0
                + ex.log(1+employment)*b_attr
                + ex.log((1+population)/area)*b_pop,
                1:
                ASC_1
                + acc_business * b_acc}
        elif p == 'education':
            V= {0:
                ASC_0
                + ex.log(1+childcare + school + higher_education)*b_attr
                + ex.log((1+population)/area)*b_pop,
                1:
                ASC_1
                + acc_education * b_acc}
        elif p == 'buy/execute':
            V= {0:
                ASC_0
                + ex.log(1+shop + medical + special_shop)*b_attr
                + ex.log((1+population)/area)*b_pop,
                1:
                ASC_1
                + globals()['acc_buy/execute'] * b_acc}
        elif p == 'leisure':
            V= {0:
                ASC_0
                + ex.log(1+daily_leisure + holiday + occasional_leisure)*b_attr
                + ex.log((1+population)/area)*b_pop,
                1:
                ASC_1
                + acc_leisure * b_acc}
        elif p == 'accompany':
            V= {0:
                ASC_0
                + ex.log(1+medical + school + childcare)*b_attr
                + ex.log((1+population)/area)*b_pop,
                1:
                ASC_1
                + acc_accompany * b_acc}
        
        # Simulate
        simulate = {j: models.logit(V, None, int(j)) for j in [0,1]}
        model = bio.BIOGEME(zones_db, simulate)
        probs = model.simulate(theBetaValues=betas[p+car_names[car]].to_dict())
        
        # Calculate generation volumes
        inner_probs[p+car_names[car]] = probs[0]
        print('{}_{} av. prob. inner: {}'.format(p,car,np.round(probs[0].mean(), 2)))

commuting_0 av. prob. inner: 0.56
business_0 av. prob. inner: 0.56
education_0 av. prob. inner: 0.56
buy/execute_0 av. prob. inner: 0.55
leisure_0 av. prob. inner: 0.57
accompany_0 av. prob. inner: 0.55
commuting_1 av. prob. inner: 0.54
business_1 av. prob. inner: 0.54
education_1 av. prob. inner: 0.55
buy/execute_1 av. prob. inner: 0.54
leisure_1 av. prob. inner: 0.55
accompany_1 av. prob. inner: 0.54


In [17]:
# If all that doesn't make sense:
# Take inner-zonal probabilities from MiD2017 data by urbanisation degree
#inner_probs = pd.read_csv(input_path + 'inner-zonal_probabilities_agg_urban.csv', index_col=0)
#inner_probs

## Compute volumes (OD matrix)

$F_{ods} = N_{os} * p^{dest}_{ods} * (1 - p^{inner}_{os})$

Indecies:
* o: origin zone
* d: destination zone
* s: demand segment

In [18]:
# Load probability results from logit step
dm = stepmodel.read_zippedpickles(model_path + 'de_destination_choice')

utility_values: 100%|████████████████████████████████████████████████████████████████████| 7/7 [00:06<00:00,  1.02it/s]


In [19]:
# Extract probability values and create new table from it
prob = dm.probabilities.set_index(['origin', 'segment']).drop(columns=['destination', 'root'])
prob.columns.name = 'destination'
volumes = prob.stack('destination').unstack('segment').reset_index()
volumes.columns.name = None
for seg in set(segments) - set(volumes.columns):
    volumes[seg] = 0
volumes.sample(2)

,origin,destination,accompany_car,accompany_no_car,business_car,business_no_car,buy/execute_car,buy/execute_no_car,commuting_car,commuting_no_car,education_car,education_no_car,leisure_car,leisure_no_car
1956600,DE27E_8,DE947_3,6.931062e-08,0.000023,2.499961e-09,7.320332e-07,6.851113e-26,1.552545e-14,9.591187e-28,7.710394e-17,7.747360e-35,4.590916e-19,0.000045,0.0
3038124,DE938_3,DE913_0,7.123169e-05,0.001559,3.978197e-03,1.065799e-02,5.195805e-05,2.868097e-03,9.093012e-05,2.615611e-03,1.068348e-05,1.385176e-03,0.001297,0.0


In [20]:
# Create inner-zonal probabilities
in_mask = volumes['origin']==volumes['destination']
if len(volumes.loc[in_mask]) == 0:
    volumes = volumes.append(pd.DataFrame({'origin': list(sm.zones.index),
                                           'destination': list(sm.zones.index)})
                            ).reset_index()
    in_mask = volumes['origin']==volumes['destination']
urban_dict = sm.zones['urbanisation'].to_dict()
for seg in segments:
    probs = inner_probs[seg]
    # Correct probabilities in volumes
    if len(probs) < len(sm.zones): # aggregated probabilities by urbanisation degree
        volumes.loc[in_mask, seg] = volumes.loc[in_mask, 'origin'].map(urban_dict).map(probs)
        volumes.loc[~in_mask, seg] = volumes.loc[~in_mask, seg] * \
                                     volumes.loc[~in_mask, 'origin'].map(urban_dict).map(1 - probs)
    else:
        volumes.loc[in_mask, seg] = volumes.loc[in_mask, 'origin'].map(probs)
        volumes.loc[~in_mask, seg] = volumes.loc[~in_mask, seg] * \
                                     volumes.loc[~in_mask, 'origin'].map(1 - probs)

In [21]:
# make sure all probabilities sum up to 1 for each zone in each segments
#volumes.groupby('origin')[segments].sum().describe()

In [22]:
# calculate number of trips
for s in segments:
    generation_dict = sm.zones[str((s, 'generation'))].to_dict()
    volumes[s] = volumes[s] * volumes['origin'].map(generation_dict)

In [23]:
# check the sum (million trips per day)
volumes[segments].sum().sum() / 1e6 / 365

157.9334806821937

In [26]:
# inter-zonal trips (billion per year)
volumes.loc[volumes['origin']!=volumes['destination'], segments].sum().sum() / 1e9

16.492485358015813

## Sparsify the OD set

Reduce the number of OD pairs to a sample while keeping the number of trips constant.

In [27]:
# Only sample, if the parameter is specified
sample_size = params['general']['od_sparse_sample']
if sample_size > 0:
    # Set a stable seed, generating the same OD set in every scenario run
    np.random.seed(42)
    # Choice probabilities of OD pairs weighted by trip volumes
    od_probabilities = volumes[segments].sum(axis=1) / volumes[segments].sum().sum()
    sample = np.random.choice(a=volumes.index,
                              size=sample_size,
                              p=od_probabilities)
    # Reduce the volumes matrix
    for seg in segments:
        expansion = volumes[seg].sum() / sample_size
        volumes[seg] = pd.Series(sample).value_counts() * expansion

In [28]:
# Check the sum (million trips per day)
volumes.loc[volumes.notna().all(axis=1), segments].sum().sum() / 365 / 1e6

157.9334806821937

In [29]:
volumes = volumes.loc[volumes.notna().all(axis=1)]

## Save

In [30]:
# Save volumes table
sm.volumes = volumes.reset_index(drop=True)
sm.to_zippedpickles(model_path + 'de_volumes', only_attributes=['volumes'])

volumes: 100%|████████████████████████████████████████████████████████████████████████| 34/34 [00:00<00:00, 218.97it/s]
